### yield, yield from, send 用法

In [1]:
def fib(n):
    index = 0
    a = 0
    b = 1

    while index < n:
        yield b
        a,b = b, a+b
        index += 1

`fib`是个生成器对象。

In [2]:
fib = fib(100)
print(fib)

<generator object fib at 0x0000000004F86EB8>


In [4]:
print(next(fib))
print(next(fib))
print(next(fib))
print(next(fib))

1
1
2
3


除了不断`next`，还可以通过`send`给迭代器传送一个值，这个值在函数中被`yield`语句返回。`send`会返回下一次迭代器`yield`的值。

In [10]:
def fib(n):
    index = 0
    a = 0
    b = 1

    while index < n:
        receive = yield b
        print('`fib` receive %d' % receive)
        a,b = b, a+b
        index += 1

fib = fib(20)
print('`fib` yield %d ' % fib.send(None))   # 效果等同于print(next(fib))
print('`fib` yield %d ' % fib.send(1))
print('`fib` yield %d ' % fib.send(1))
print('`fib` yield %d ' % fib.send(1))
print('`fib` yield %d ' % fib.send(1))

`fib` yield 1 
`fib` receive 1
`fib` yield 1 
`fib` receive 1
`fib` yield 2 
`fib` receive 1
`fib` yield 3 
`fib` receive 1
`fib` yield 5 


迭代器可以嵌套。用`yield from`可以套一个迭代器。

In [8]:
def fun_inner():
    i = 0
    while True:
        i = yield i

def fun_outer():
    yield from fun_inner()

outer = fun_outer()
outer.send(None)
for i in range(5):
    print(outer.send(i))

0
1
2
3
4


使用迭代器实现协程。

In [12]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] Consuming %s...' % n)
        r = '200 OK'

def produce(c):
    c.send(None)
    n = 0
    while n < 5:
        n = n + 1
        print('[PRODUCER] Producing %s...' % n)
        r = c.send(n)
        print('[PRODUCER] Consumer return: %s' % r)
    c.close()

c = consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return: 200 OK


使用`async`, `await`关键字实现协程。
参考`demo_coroutine.py`和`demo_coroutine_producer_consumer.py`